# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Jiaxin Hu

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Functions that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [6]:
# Model Inputs
x =tf.placeholder(tf.float32,[None,784]) ### MNIST images enter graph here ###
y_ =tf.placeholder(tf.float32, shape=[None, 10])  ### MNIST labels enter graph here ###

# Define the graph


### Create your MLP here##
in_puts=784 
h1_units=500

W_1=weight_variable([in_puts,h1_units])
b_1=bias_variable([h1_units])

W_2=weight_variable([h1_units,10])
b_2=bias_variable([10])

hidden_1=tf.nn.relu(tf.matmul(x,W_1)+b_1)


y_mlp=tf.nn.softmax(tf.matmul(hidden_1,W_2)+b_2)


### Make sure to name your MLP output as y_mlp ###



# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.084
step 250, validation accuracy 0.814
step 500, validation accuracy 0.85
step 750, validation accuracy 0.873
step 1000, validation accuracy 0.929
step 1250, validation accuracy 0.942
step 1500, validation accuracy 0.941
step 1750, validation accuracy 0.959
step 2000, validation accuracy 0.949
step 2250, validation accuracy 0.958
step 2500, validation accuracy 0.96
step 2750, validation accuracy 0.969
step 3000, validation accuracy 0.96
step 3250, validation accuracy 0.973
step 3500, validation accuracy 0.959
step 3750, validation accuracy 0.966
test accuracy 0.9619


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

There is a great difference in accuracy between the two results of sigmoid and ReLU. When we use sigmoid, the final test accuracy is about 0.8424; when we use ReLU function, the final test accuracy reach about 0.9619. Therefore, as these two function take almost the same time to run, the ReLU one is obviously more effcient than the sigmoid one.

***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [2]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [ ]:
# Model Inputs
x =tf.placeholder(tf.float32,[None,784]) ### MNIST images enter graph here ###
y_ =tf.placeholder(tf.float32, shape=[None, 10]) ### MNIST labels enter graph here ###

# Define the graph


### Create your CNN here##
x_image=tf.reshape(x,[-1,28,28,1])

W_conv_1=weight_variable([5,5,1,32])
b_conv_1=bias_variable([32])
h_conv_1=tf.nn.relu(conv2d(x_image,W_conv_1)+b_conv_1)
h_pool_1=max_pool_2x2(h_conv_1)

W_conv_2=weight_variable([5,5,32,64])
b_conv_2=bias_variable([64])
h_conv_2=tf.nn.relu(conv2d(h_pool_1,W_conv_2)+b_conv_2)
h_pool_2=max_pool_2x2(h_conv_2)

W_fc_1=weight_variable([7*7*64,1024])
b_fc_1=bias_variable([1024])
h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64])              #reshape成向量
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)


W_fc_2=weight_variable([1024,10])
b_fc_2=bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc_1,W_fc_2)+b_fc_2)
### Make sure to name your CNN output as y_conv ###



# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.096
step 250, validation accuracy 0.638
step 500, validation accuracy 0.764
step 750, validation accuracy 0.858
step 1000, validation accuracy 0.852
step 1250, validation accuracy 0.968
step 1500, validation accuracy 0.976
step 1750, validation accuracy 0.978
step 2000, validation accuracy 0.976
step 2250, validation accuracy 0.99
step 2500, validation accuracy 0.984
step 2750, validation accuracy 0.984
step 3000, validation accuracy 0.976
step 3250, validation accuracy 0.964
step 3500, validation accuracy 0.978
step 3750, validation accuracy 0.984
step 4000, validation accuracy 0.994
step 4250, validation accuracy 0.99
step 4500, validation accuracy 0.99
step 4750, validation accuracy 0.972
step 5000, validation accuracy 0.972
step 5250, validation accuracy 0.99
step 5500, validation accuracy 0.98
step 5750, validation accuracy 0.986
step 6000, validation accuracy 0.992
step 6250, validation accuracy 0.986
step 6500, validation accuracy 0.988
step 6750, v

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

Obviously, CNN is better than MLP on the test accuracy whenever we use sigmoid or ReLU. Even though when CNN only runs same steps as MLP, CNN perform much better. However, the training time of CNN is much longer than MLP.To illustrate, MLP only use less than 1 minute to finish 4000 steps while CNN use about 15 minutes to repeat 4000 times. And as far as I am concerned, I would like to prefer CNN than MLP for the accuracy of CNN model can be rised up rapidly.Though we also can get a good result when we repeat MLP 10000 times, we can get a better accuracy(training result) in less steps with CNN.


***